In [ ]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from dmd_toolbox.pattern import analogDMDPattern, binaryMask
from dmd_toolbox.pattern.dataloader import DMDPadding, analogBWImage
from dmd_toolbox.pattern.func import gaussian
from dmd_toolbox.utils import checkdep_usetex

In [ ]:
matplotlib.rcParams['text.usetex'] = checkdep_usetex(True)
ROOT = Path("./..")

## Usage for ED (on Cicero)

In [ ]:
cicero_image = analogBWImage(img_path=ROOT.joinpath("Test_Analog_Images/Cicero_BW.jpg"))

# normalise cicero_image to 0-0.9 range (image is encoded in 8 bits, so values are between 0 and 255)

DMD_height, DMD_width = 1080, 1920

# Setup the padding for the Cicero image on the DMD array
padding_cicero_dim = (
    cicero_image.shape[0] // 2,  # Top padding
    DMD_height
    - cicero_image.shape[0] // 2
    - cicero_image.shape[0],  # Bottom padding
    cicero_image.shape[1] // 2,  # Left padding
    DMD_width
    - cicero_image.shape[1] // 2
    - cicero_image.shape[1],  # Right padding
)
padding_cicero = DMDPadding(padding_cicero_dim, DMD_height, DMD_width)

# Crreate an analogDMDPattern instance for the Cicero image and correct for distortion
cicero_pattern = analogDMDPattern(cicero_image)
# Create a binary mask from the cicero pattern (assuming A = 0.5, B = 0.5 for calibration)
cicero_mask = binaryMask(
    cicero_pattern.pattern, padding_cicero_dim, DMD_height, DMD_width
)
cicero_mask.calibrate_pattern(0.5, 0.5)

cicero_mask.perform_error_diffusion(
    cicero_image, save_path="ED Patterns/Cicero_test/Cicero_image_ED.bmp"
)

fig, ax = plt.subplots(2, 2, figsize=(20, 20))
ax[0, 0].imshow(cicero_image, cmap="gray")
ax[0, 0].set_title("Original Cicero Image")
ax[0, 0].axis("off")

ax[0, 1].imshow(cicero_mask.analog_pattern.pattern_corr, cmap="gray")
ax[0, 1].set_title(
    r"Corrected Cicero Image (analogDMDPattern) $\alpha="
    + f"{np.rad2deg(-4.5):.2f}$,"
    + r"$\beta="
    + f"{np.rad2deg(-2.5):.2f}$,"
    + r"$\gamma=1.09$",
)
ax[0, 1].axis("off")

ax[1, 0].imshow(cicero_mask.calibrated_pattern, cmap="gray")
ax[1, 0].set_title("Calibrated Cicero Pattern")
ax[1, 0].axis("off")

ax[1, 1].imshow(cicero_mask.binary_mask, cmap="gray")
ax[1, 1].set_title("Binary Mask after Error Diffusion")
ax[1, 1].axis("off")

plt.tight_layout()
plt.savefig(ROOT.joinpath("ED Patterns/Cicero_test/cicero_patterns.png"), dpi=300, bbox_inches="tight")
# plt.show()


## Usage for distortion correction - Gaussian Potential

In [ ]:

height, width = 1080, 1920
x, y = np.meshgrid(np.linspace(-width//2, width//2, width), np.linspace(-height//2, height//2, height))
gaussian_potential = gaussian(np.zeros((height, width)), sigma=100)

# Create a analogDMDpattern instance
gaussian_pattern = analogDMDPattern(gaussian_potential)

# Correct for distortion with gaussian_corr
alpha, beta, gamma = np.deg2rad(-4.5), np.deg2rad(-2.5), 1.09
gaussian_pattern.correct_distortion(alpha, beta, gamma)

fig, ax = plt.subplots(1, 2, figsize=(10, 20))
ax[0].imshow(gaussian_potential, cmap='gray')
ax[0].set_title("Original Gaussian Potential")
ax[0].axis('off')

ax[1].imshow(gaussian_pattern.pattern_corr, cmap='gray')
ax[1].set_title(r"Corrected Gaussian Potential (analogDMDPattern) $\alpha="
                + f"{np.rad2deg(alpha):.2f}$," +r"$\beta="+f"{np.rad2deg(beta):.2f}$,"
                + r"$\gamma="+f"{gamma:.2f}$")
ax[1].axis('off')
